## TODO

#### coding
- Adopt standard variable convention for op, op_str, etc.
    - matrixoperators are given name mat_op, singletrace are given st_op
    - op string names 'X', 'P' are op_str
    - operator tuples are op

- how to handle 'empty' operator SingleTraceOperator(data={():0})
- related, but maybe distinct: how to handle 'zero' operator like A - A = 0?

#### constraints
- inspect constraints, make sure they look good
- I'm confused about the reality constraint, and the fact that the expectation values are either real or imaginary.
- For the models studied in Han et al, expectation values of odd-degree O's vanish. This does not seem to be imposed by the linear constraints. Does it come from the quadratic ones? Or is it a discrete symmetry that needs to be separately imposed?

### efficient implementation
- can I reduce matrix sizes by discarding redundant constraints?
- can I make my bootstrap matrix block diagonal?
- use sparse representations as early as possible

## TODO - Tues May 28
- be more carful about reality constraints
- how to accomodate the fact that <tr(XP)> is purely imaginary and satisfies <tr(XP)> = <tr(PX)>*?
- from looking at Han et al, I might be missing some cyclic constraints that are linear only.
- revisit how the linear constraints are being added incrementally

In [1]:
from typing import Union, Self
from numbers import Number
from itertools import chain, product

import numpy as np
import sympy as sp
import cvxpy as cp

import scipy
from scipy.sparse import csr_matrix
import scipy.sparse as sparse
from scipy.linalg import qr
from scipy.sparse import coo_matrix, csc_matrix
from scipy.sparse.linalg import splu, svds
from sksparse.cholmod import cholesky

from bmn.algebra import MatrixOperator, SingleTraceOperator, MatrixSystem
from bmn.linear_algebra import get_null_space, create_sparse_matrix_from_dict, is_in_row_space, get_row_space
from bmn.bootstrap import BootstrapSystem
from bmn.solver import minimal_eigval, sdp_init, sdp_relax, sdp_minimize, minimize, get_quadratic_constraint_vector

np.set_printoptions(linewidth=120)  # Adjust the number to the desired width

## One Matrix Model

In [2]:
matrix_system = MatrixSystem(
    #operator_basis=['X', 'P'],
    operator_basis=['X', 'Pi'],
    commutation_rules_concise = {
        #('P', 'X'): -1j,
        ('Pi', 'X'): 1, # use Pi' = i P to ensure reality
    },
    #hermitian_dict={'P': True, 'X': True},
    hermitian_dict={'Pi': False, 'X': True},
)

# scale variables as P = sqrt(N) P', X = sqrt(N) X'
hamiltonian = SingleTraceOperator(
        #data={("P", "P"): 1, ("X", "X"): 1, ("X", "X", "X", "X"): 7}
        data={("Pi", "Pi"): -1, ("X", "X"): 1, ("X", "X", "X", "X"): 0.01}
    )

# <tr G O > = 0 might need to be applied only for O with deg <= L-2
#gauge = MatrixOperator(data={('X', 'P'): 1j, ('P', 'X'): -1j, ():1})
gauge = MatrixOperator(data={('X', 'Pi'): 1, ('Pi', 'X'): -1, ():1})

bootstrap = BootstrapSystem(
    matrix_system=matrix_system,
    hamiltonian=hamiltonian,
    gauge=gauge,
    half_max_degree=2
)

bootstrap.get_null_space_matrix();

Assuming all operators are either Hermitian or anti-Hermitna.
Null matrix has not been computed yet. Building it by solving the linear constraints.
generated 9 Hamiltonian constraints
generated 7 gauge constraints
generated 21 reality constraints
generated 10 odd degree vanish constraints


In [3]:
bootstrap.param_dim, bootstrap.param_dim_null

(31, 8)

In [4]:
bootstrap.build_bootstrap_table()

<49x8 sparse matrix of type '<class 'numpy.float64'>'
	with 176 stored elements in Compressed Sparse Row format>

In [5]:
param = np.random.normal(0, 1, size=bootstrap.param_dim_null)
bootstrap_matrix = (bootstrap.build_bootstrap_table() @ param).reshape((bootstrap.psd_matrix_dim, bootstrap.psd_matrix_dim))
print(bootstrap_matrix)

[[ 0.47710894  0.          0.          0.53064329 -0.23855447  0.23855447 -0.54000761]
 [ 0.          0.53064329 -0.23855447  0.          0.          0.          0.        ]
 [ 0.         -0.23855447  0.54000761  0.          0.          0.          0.        ]
 [ 0.53064329  0.          0.          0.46821573  0.92838732  1.45903062  0.75146442]
 [-0.23855447  0.          0.          0.92838732 -0.33507169 -0.57362616  0.59140748]
 [ 0.23855447  0.          0.          1.45903062 -0.57362616 -0.33507169  0.05139988]
 [-0.54000761  0.          0.          0.75146442  0.59140748  0.05139988  1.13283775]]


In [6]:
minimal_eigval(bootstrap.build_bootstrap_table(), param)

-2.099100394601347

In [7]:
import torch
import torch.optim as optim
from torch.nn import ReLU

# build the Ax = b constraint
Avec = bootstrap.single_trace_to_coefficient_vector(SingleTraceOperator(data={(): 1}), return_null_basis=True)
Avec = torch.from_numpy(Avec).type(torch.float)

# Hamiltonian vector
Hvec = bootstrap.single_trace_to_coefficient_vector(bootstrap.hamiltonian, return_null_basis=True)
Hvec = torch.from_numpy(Hvec).type(torch.float)

# build the bootstrap array
bootstrap_array_sparse = bootstrap.build_bootstrap_table().todense()
bootstrap_array_torch = torch.from_numpy(bootstrap_array_sparse).type(torch.float)

# build the constraints
quadratic_constraints = bootstrap.build_quadratic_constraints()
quadratic_constraint_linear = torch.from_numpy(quadratic_constraints['linear']).type(torch.float)
quadratic_constraint_quadratic = torch.from_numpy(quadratic_constraints['quadratic']).type(torch.float)

def energy(param):
    return Hvec @ param

def quadratic_loss(param):
    quadratic_constraints = (
        torch.einsum('Iab, a, b -> I', quadratic_constraint_quadratic, param, param)
        + torch.einsum('Ia, a -> I', quadratic_constraint_linear, param)
    )
    quadratic_constraints = torch.sum(torch.square(quadratic_constraints))
    return quadratic_constraints

def Axb_loss(param):
    return torch.square(Avec @ param - 1)

def psd_loss(param):
    bootstrap_matrix = (bootstrap_array_torch @ param).reshape((7, 7)) # is this reshaping correct?
    smallest_eigv = torch.linalg.eigvalsh(bootstrap_matrix)[0]
    return ReLU()(-smallest_eigv)

def build_loss(param):
    lambda_psd = 10
    lambda_quadratic = 10
    lambda_Axb = 10
    #loss = energy(param) + lambda_psd * psd_loss(param) + lambda_quadratic * quadratic_loss(param) + lambda_Axb * Axb_loss(param)
    return quadratic_loss(param)# + Axb_loss(param)

constraint from operator_idx = 17 is quadratically trivial.
constraint from operator_idx = 28 is quadratically trivial.


In [ ]:
param = torch.randn(bootstrap.param_dim_null)


In [31]:
#param = torch.randn(bootstrap.param_dim_null, requires_grad=True)
#optimizer = optim.Adam([param], lr=0.01)

param = torch.randn(bootstrap.param_dim_null - 1, requires_grad=True)
optimizer = optim.Adam([param], lr=1e-3)

# Training loop
num_epochs = 10_000
for epoch in range(num_epochs):
    optimizer.zero_grad()  # Clear previous gradients

    param0 = (1.0 - Avec[1:] @ param) / Avec[0]
    param_big = torch.cat((torch.Tensor([param0]), param), dim=-1)
    loss = quadratic_loss(param_big) + param0**2

   #loss = build_loss(param_big)  # Compute the loss
    loss.backward()  # Compute gradients
    optimizer.step()  # Update the parameters

    # Print the loss for monitoring
    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')


Epoch 10/10000, Loss: 20.207059860229492
Epoch 20/10000, Loss: 20.076557159423828
Epoch 30/10000, Loss: 19.936912536621094
Epoch 40/10000, Loss: 19.789813995361328
Epoch 50/10000, Loss: 19.643617630004883
Epoch 60/10000, Loss: 19.504222869873047
Epoch 70/10000, Loss: 19.37131690979004
Epoch 80/10000, Loss: 19.242462158203125
Epoch 90/10000, Loss: 19.116897583007812
Epoch 100/10000, Loss: 18.99535369873047
Epoch 110/10000, Loss: 18.878307342529297
Epoch 120/10000, Loss: 18.76557159423828
Epoch 130/10000, Loss: 18.656871795654297
Epoch 140/10000, Loss: 18.552207946777344
Epoch 150/10000, Loss: 18.45161247253418
Epoch 160/10000, Loss: 18.355045318603516
Epoch 170/10000, Loss: 18.262428283691406
Epoch 180/10000, Loss: 18.1737117767334
Epoch 190/10000, Loss: 18.088848114013672
Epoch 200/10000, Loss: 18.00778579711914
Epoch 210/10000, Loss: 17.930465698242188
Epoch 220/10000, Loss: 17.856834411621094
Epoch 230/10000, Loss: 17.786827087402344
Epoch 240/10000, Loss: 17.72039031982422
Epoch 250

In [30]:
param0

tensor(-24.9797, grad_fn=<DivBackward0>)

In [28]:
Avec @ param_big

tensor(0.1000, grad_fn=<DotBackward0>)

In [27]:
energy(param_big), quadratic_loss(param_big), Axb_loss(param_big), psd_loss(param_big)

(tensor(-0.0723, grad_fn=<DotBackward0>),
 tensor(0.0574, grad_fn=<SumBackward0>),
 tensor(0.8100, grad_fn=<PowBackward0>),
 tensor(1.3996, grad_fn=<ReluBackward0>))

In [ ]:
if not np.allclose(np.imag(bootstrap_matrix), np.zeros_like(bootstrap_matrix)):
    raise ValueError("Bootstrap matrix is not real.")

bootstrap_matrix = np.real(bootstrap_matrix)

if not np.allclose((bootstrap_matrix - bootstrap_matrix.T), np.zeros_like(bootstrap_matrix)):
    raise ValueError("Bootstrap matrix is not symmetric.")

In [ ]:
np.linalg.eigvals(bootstrap_matrix)

In [ ]:
# testing
linear_constraint_matrix = bootstrap.build_linear_constraints().todense()

op = SingleTraceOperator(data={(): 1})
param_vector = bootstrap.single_trace_to_coefficient_vector(op)
param_vector_null = bootstrap.single_trace_to_coefficient_vector(op, return_null_basis=True)
param_vector_null, is_in_row_space(matrix=linear_constraint_matrix, vector=param_vector)

In [ ]:
op = SingleTraceOperator(data={('Pi'): 1})
param_vector = bootstrap.single_trace_to_coefficient_vector(op)
param_vector_null = bootstrap.single_trace_to_coefficient_vector(op, return_null_basis=True)
param_vector_null, is_in_row_space(matrix=linear_constraint_matrix, vector=param_vector)

In [ ]:
op = SingleTraceOperator(data={('X'): 1})
param_vector = bootstrap.single_trace_to_coefficient_vector(op)
param_vector_null = bootstrap.single_trace_to_coefficient_vector(op, return_null_basis=True)
param_vector_null, is_in_row_space(matrix=linear_constraint_matrix, vector=param_vector)

In [ ]:
op = SingleTraceOperator(data={('Pi', 'Pi', 'Pi'): 1})
param_vector = bootstrap.single_trace_to_coefficient_vector(op)
param_vector_null = bootstrap.single_trace_to_coefficient_vector(op, return_null_basis=True)
param_vector_null, is_in_row_space(matrix=linear_constraint_matrix, vector=param_vector)

In [ ]:
# check that the quadratic matrix is symmetric
quadratic_constraints = bootstrap.build_quadratic_constraints()
np.sum(np.abs(quadratic_constraints['quadratic'] - np.einsum('Iab->Iba', quadratic_constraints['quadratic'])))

In [ ]:
cyclic_constraints = bootstrap.generate_cyclic_constraints()
cyclic_constraints[17]

In [ ]:
param_optimized = minimize(
    bootstrap=bootstrap,
    op=bootstrap.hamiltonian,
    init=100*np.random.normal(size=bootstrap.param_dim_null),
    maxiters=25,
    eps=5e-1,
    reg=5e-5,
)

In [ ]:
# check the quadratically-trivial quadratic constraints
vec = bootstrap.single_trace_to_coefficient_vector(
    st_operator=cyclic_constraints[17]['lhs'],
    return_null_basis=True
    )
vec.dot(param_optimized)

In [ ]:
# check the quadratically-trivial quadratic constraints
vec = bootstrap.single_trace_to_coefficient_vector(
    st_operator=cyclic_constraints[28]['lhs'],
    return_null_basis=True
    )
vec.dot(param_optimized)

In [ ]:
get_quadratic_constraint_vector(quadratic_constraints=quadratic_constraints, param_vector=param_optimized)

In [ ]:
bootstrap_array_sparse = bootstrap.build_bootstrap_table()
minimal_eigval(bootstrap_array_sparse=bootstrap_array_sparse, parameter_vector_null=param_optimized)

In [ ]:
op = bootstrap.hamiltonian
vec = bootstrap.single_trace_to_coefficient_vector(op, return_null_basis=True)
vec.dot(param_optimized)

In [ ]:
op = SingleTraceOperator(
        data={("X", "X"): 1}
    )
vec = bootstrap.single_trace_to_coefficient_vector(op, return_null_basis=True)
vec.dot(param_optimized)

In [ ]:
op = SingleTraceOperator(
        data={("X", "X", "X", "X"): 1}
    )
vec = bootstrap.single_trace_to_coefficient_vector(op, return_null_basis=True)
vec.dot(param_optimized)

Things to look into:
- inspect the quadratic constraints and make sure they make sense
    - are any linear?
- can I initialize such that Ax = b?
    - yes, but doesn't change outcome
    - why is Ax = b not being imposed?
- what if I impose that odd degree terms vanish directly?
- should be able to reproduce known results for g->0

In [ ]:
op_cons=[SingleTraceOperator(data={(): 1})]
A_op = sparse.csr_matrix((0, bootstrap.param_dim_null))
b_op = np.zeros(0)
for op in op_cons:
    A_op = sparse.vstack(
        [
            A_op,
            sparse.csr_matrix(
                bootstrap.single_trace_to_coefficient_vector(
                    op, return_null_basis=True
                )
            ),
        ]
    )
    b_op = np.append(b_op, 1)

In [ ]:
A_op.shape, b_op.shape

In [ ]:
A_op @ param_optimized,  b_op

In [ ]:
bootstrap_array_sparse = bootstrap.build_bootstrap_table()

In [ ]:
param_init = sdp_init(
    bootstrap_array_sparse,
    A=A_op,
    b=b_op,
    init=np.random.normal(size=bootstrap.param_dim_null),
    reg=0,
    maxiters=5_000,
    eps=1e-4,
    verbose=True
    )

In [ ]:
(bootstrap.null_space_matrix @ param_init)[0]

In [ ]:
(bootstrap.null_space_matrix @ param_optimized)[0]

In [ ]:
param_optimized

In [ ]:
for i in range(bootstrap.param_dim):
    val = (bootstrap.null_space_matrix @ param_optimized)[i]
    if abs(val) < 1e-5:
        print(bootstrap.operator_list[i], val)

In [ ]:
for i in range(bootstrap.param_dim):
    val = (bootstrap.null_space_matrix @ param_optimized)[i]
    print(bootstrap.operator_list[i], val)

In [ ]:
bootstrap.generate_cyclic_constraints()

In [ ]:
bootstrap.param_dim, bootstrap.param_dim_null

In [ ]:
np.einsum('Iij, i, j -> I', quadratic_constraints['quadratic'], param_optimized, param_optimized)

In [ ]:
quadratic_constraints['linear'] @ param_optimized + np.einsum('Iij, i, j -> I', quadratic_constraints['quadratic'], param_optimized, param_optimized)

In [ ]:
bootstrap_matrix = (bootstrap.build_bootstrap_table() @ param_optimized).reshape((bootstrap.psd_matrix_dim, bootstrap.psd_matrix_dim))
print(bootstrap_matrix[0:4, 0:4])

In [ ]:
scipy.linalg.eigvalsh(bootstrap_matrix)[0]

Revisit quadratic constraints - Han et al has some of them being linearly only...

In [ ]:
bootstrap.build_linear_constraints(return_matrix=False)

In [ ]:
(bootstrap.gauge * MatrixOperator(data={('P', 'P'): 1})).trace()

In [ ]:
bootstrap.null_space_matrix[2]

In [ ]:
bootstrap.operator_list[0:bootstrap.psd_matrix_dim]

In [ ]:
param = np.random.normal(0, 1, size=bootstrap.param_dim_null)
bootstrap_matrix = (bootstrap.build_bootstrap_table() @ param).reshape((bootstrap.psd_matrix_dim, bootstrap.psd_matrix_dim))
print(bootstrap_matrix[0:4, 0:4])

In [ ]:
bootstrap.build_bootstrap_table()

In [ ]:
(bootstrap_matrix - bootstrap_matrix.T.conj())

In [ ]:
bootstrap.operator_list[: bootstrap.psd_matrix_dim]

In [ ]:
param = minimize(
    bootstrap=bootstrap,
    op=hamiltonian,
    init=np.zeros(bootstrap.param_dim_null),
    )

In [ ]:
param

In [ ]:
bootstrap.null_space_matrix @ param

In [ ]:
quadratic_constraints = bootstrap.build_quadratic_constraints()
get_quadratic_constraint_vector(
    quadratic_constraints=quadratic_constraints,
    param_vector=param
    )

In [ ]:
bootstrap_matrix = (bootstrap.build_bootstrap_table() @ param).reshape((bootstrap.psd_matrix_dim, bootstrap.psd_matrix_dim))
np.linalg.eigvals(bootstrap_matrix)

In [ ]:
bootstrap_matrix - bootstrap_matrix.T

In [ ]:
bootstrap.generate_hamiltonian_constraints()

In [ ]:
bootstrap.generate_gauge_constraints()

In [ ]:
bootstrap.generate_reality_constraints()

In [ ]:
from collections import Counter

param = 1j * np.zeros(bootstrap.param_dim_null)
seen_indices = []
for op, idx in bootstrap.operator_dict.items():
    if idx == 0:
        param[idx] = 1
        seen_indices.append(idx)
    else:
        op_dagger = op[::-1]
        idx_dagger = bootstrap.operator_dict[op_dagger]
        if idx not in seen_indices:
            if Counter(op).get('P', 0) % 2 == 0:
                param[idx] = np.random.normal()
                param[idx_dagger] = param[idx]
            else:
                param[idx] = 1j * np.random.normal()
                param[idx_dagger] = - param[idx]
        seen_indices.extend([idx, idx_dagger])

In [ ]:
# testing
op = SingleTraceOperator(data={(): 1})
param_vector = bootstrap.single_trace_to_coefficient_vector(op)
param_vector_null = bootstrap.single_trace_to_coefficient_vector(op, return_null_basis=True)
param_vector_null, is_in_row_space(matrix=linear_constraint_matrix, vector=param_vector)

In [ ]:
(
    bootstrap.generate_gauge_constraints()[0],
    #bootstrap.generate_reality_constraints()[0],
    bootstrap.generate_hamiltonian_constraints()[3],
)

In [ ]:
linear_constraint_matrix @ param_vector

In [ ]:
linear_constraint_matrix[8]

In [ ]:
param_vector

In [ ]:
row_space_matrix = get_row_space(matrix=linear_constraint_matrix)

In [ ]:
row_space_matrix @ param_vector

In [ ]:
c, residuals, rank, s = np.linalg.lstsq(row_space_matrix.T, param_vector, rcond=None)

# Reconstruct v from the row space basis using the coefficients c
v_projected = row_space_matrix.T @ c
v_projected

In [ ]:
[i for i, c_val in enumerate(c) if np.abs(c_val) > 1e-10]

In [ ]:
is_in_row_space(matrix=linear_constraint_matrix, vector=param_vector)

## OLD

In [ ]:
nu = 1.0

matrix_system = MatrixSystem(
    operator_basis=['X0', 'X1', 'X2', 'P0', 'P1', 'P2'],
    commutation_rules_concise = {
        ('P0', 'X0'): -1j,
        ('P1', 'X1'): -1j,
        ('P2', 'X2'): -1j,
    }
)

# scale variables as P = sqrt(N) P', X = sqrt(N) X'
hamiltonian = SingleTraceOperator(
        data={
           # kinetic term
            ("P0", "P1"): 1,
            ("P1", "P1"): 1,
            ("P1", "P1"): 1,
            # quadratic term
            ('X0', 'X0'): nu**2 / 2,
            ('X1', 'X1'): nu**2 / 2,
            ('X2', 'X2'): nu**2 / 2,
            # cubic term
            ('X0', 'X1', 'X2'): 6 * 1j * nu,
            # quadratic term (XY)
            ('X0', 'X1', 'X0', 'X1'): - 1/4,
            ('X1', 'X0', 'X1', 'X0'): -1/4,
            ('X0', 'X1', 'X1', 'X0'): 1/4,
            ('X1', 'X0', 'X0', 'X1'): 1/4,
            # quadratic term (XZ) TODO check sign
            ('X0', 'X2', 'X0', 'X2'): - 1/4,
            ('X2', 'X0', 'X2', 'X0'): -1/4,
            ('X0', 'X2', 'X2', 'X0'): 1/4,
            ('X2', 'X0', 'X0', 'X2'): 1/4,
            # quadratic term (YZ)
            ('X1', 'X2', 'X1', 'X2'): - 1/4,
            ('X2', 'X1', 'X2', 'X1'): -1/4,
            ('X1', 'X2', 'X2', 'X1'): 1/4,
            ('X2', 'X1', 'X1', 'X2'): 1/4,
            }
            )

# <tr G O > = 0 might need to be applied only for O with deg <= L-2
gauge = MatrixOperator(
    data={
        ('X0', 'P0'): 1j,
        ('P0', 'X0'): -1j,
        ('X1', 'P1'): 1j,
        ('P1', 'X1'): -1j,
        ('X2', 'P2'): 1j,
        ('P2', 'X2'): -1j,
        ():1}
    )

bootstrap = BootstrapSystem(
    matrix_system=matrix_system,
    hamiltonian=hamiltonian,
    gauge=gauge,
    half_max_degree=2
)
#bootstrap.operator_list[:bootstrap.psd_matrix_dim]
bootstrap.matrix_system.commutation_rules

In [ ]:
bootstrap.build_linear_constraints()